## Quantum Programming Project - Modular Exponentiation

Project-03

- Lea Jesenkovic
- Lars Herbold

In [5]:
from qiskit import QuantumCircuit
from logic_gates import LogicGatesExtension    

### MONKEY PATCHING ###
QuantumCircuit.and_gate = LogicGatesExtension.apply_and
QuantumCircuit.or_gate = LogicGatesExtension.apply_or
QuantumCircuit.xor_gate = LogicGatesExtension.apply_xor
QuantumCircuit.controlled_and_gate = LogicGatesExtension.apply_controlled_and
QuantumCircuit.controlled_or_gate = LogicGatesExtension.apply_controlled_or
QuantumCircuit.controlled_xor_gate = LogicGatesExtension.apply_controlled_xor

def set_bits(circuit, A, X):
    """
    Initializes the bits of register A with the binary string X.
    - circuit: The QuantumCircuit object
    - A: List of qubit indices
    - X: Binary string (e.g., '01011')
    """
    for i, bit in enumerate(X):
        if bit == '1':
            circuit.x(A[i])

def copy(circuit, A, B):
    """
    Copies the binary string from register A to register B.
    Assumes len(A) == len(B) and B is initialized to |0>.
    """
    for i in range(len(A)):
        # Apply CNOT: control is A[i], target is B[i]
        circuit.cx(A[i], B[i])

# TODO Look over it again
def full_adder(circuit, a, b, r, c_in, c_out, AUX):
    """
    Implements a full adder.
    Logic:
    r = a ^ b ^ c_in
    c_out = (a & b) | (c_in & (a ^ b))
    """
    # 1. Calculate (a ^ b) and store temporarily in r
    circuit.xor_gate(a, b, r)
    
    # 2. Calculate c_out part 1: (c_in AND (a ^ b)) 
    # Use AUX[0] as temporary storage for the AND result
    circuit.and_gate(c_in, r, AUX[0])
    
    # 3. Finalize r: r = (a ^ b) ^ c_in
    circuit.cx(c_in, r)
    
    # 4. Calculate c_out part 2: (a AND b)
    # Use AUX[1] for this temporary result
    circuit.and_gate(a, b, AUX[1])
    
    # 5. Finalize c_out: OR the two partial results from AUX
    circuit.or_gate(AUX[0], AUX[1], c_out)
    
    # 6. CLEANUP: Reset AUX register to |0> for qubit reuse
    # Re-applying AND gates resets the targets since they are reversible
    circuit.and_gate(a, b, AUX[1])
    circuit.and_gate(c_in, r, AUX[0]) # Note: r was modified, use logic carefully here

def add(circuit, A, B, R, AUX):
    """
    Adds number(A) to number(B) and stores the result in register R.
    A, B, R: Lists of qubit indices.
    AUX: List of qubit indices for carry bits and internal full_adder logic.
    """
    n = len(A)
    
    # We need a carry-in (c_in) and a carry-out (c_out) for each bit.
    # For bit 'i', c_in is AUX[i] and c_out is AUX[i+1].
    # Let's reserve the first few qubits of AUX for carries and 
    # the rest for the full_adder's internal temporary workspace.
    carries = AUX[:n+1] 
    fa_internal_aux = AUX[n+1:] 

    for i in range(n):
        # Apply full_adder for each bit position
        # inputs: A[i], B[i], carries[i] (c_in)
        # outputs: R[i] (result), carries[i+1] (c_out)
        circuit.full_adder(
            A[i], 
            B[i], 
            R[i], 
            carries[i], 
            carries[i+1], 
            fa_internal_aux
        )

def subtract(circuit, A, B, R, AUX):
    """
    Subtracts Number(B) from Number(A) and stores the result in R[cite: 107].
    Logic: A - B = A + (NOT B) + 1.
    """
    n = len(A)
    carries = AUX[:n+1]
    fa_internal_aux = AUX[n+1:]

    # 1. Negate each bit in B 
    for b_qubit in B:
        circuit.x(b_qubit)

    # 2. Set the first carry-in bit to 1 
    circuit.x(carries[0])

    # 3. Apply the adder circuit cascade 
    for i in range(n):
        circuit.full_adder(
            A[i], 
            B[i], 
            R[i], 
            carries[i], 
            carries[i+1], 
            fa_internal_aux
        )

    # 4. CLEANUP: Reset B and the first carry-in to original state
    # (Note: AUX carries are usually uncomputed in modular steps)
    circuit.x(carries[0])
    for b_qubit in B:
        circuit.x(b_qubit)

def greater_or_eq(circuit, A, B, r, AUX):
    """
    Tests whether A >= B and stores result in r[cite: 141].
    """
    n = len(A)
    # R must be initialized to |0> for the subtraction result
    # We can use a part of AUX as a temporary R register
    temp_R = AUX[n+1 : 2*n+1] 
    
    # 1. Perform subtraction
    circuit.subtract(A, B, temp_R, AUX)
    
    # 2. The final carry bit (AUX[n]) determines the result 
    circuit.cx(AUX[n], r)
    
    # 3. CLEANUP: Uncompute subtraction to reset AUX and temp_R to |0>
    circuit.subtract(A, B, temp_R, AUX)

# TEST FUNCTION USE
# qc = QuantumCircuit(4)
# qc.and_gate(0, 1, 2)
# qc.barrier()
# qc.or_gate(0, 1, 3)
# qc.barrier()
# qc.xor_gate(0,1,2)



print(qc.draw())

          
q_0: ─────
          
q_1: ─────
          
q_2: ─────
          
q_3: ─────
     ┌───┐
q_4: ┤ X ├
     ├───┤
q_5: ┤ X ├
     └───┘
q_6: ─────
     ┌───┐
q_7: ┤ X ├
     └───┘
